# Views
## Definition
- A saved `SELECT` statement
- A view can be seen as a **virtual table** composed of other tables & views
- **No data is stored in the view itself**

## Used for
- Hide complexity of the database
- Hide complex database design make large and complex queries accessible and reusable
- Can be used as a partial solution for complex problems
- Used for securing data access: revoke access to tables and grant access to customised views. 
- Organise data for export to other applications

## Caveats
-  At each referral the underlying SELECT is **re-executed**, which could cause performance issues of it's overused.
- `ORDER BY` in a `View` is ... **worthless**.

## Creating a view




In [ ]:
CREATE VIEW vw_product_quantity_sold_to_customer AS
-- Nothing new from this point, a simple SELECT Statement.
SELECT 
 ProductId
,CustomerName
,SUM(Quantity) AS 'Total Amount' 
FROM Customer 
    JOIN Orders [Order]             ON [Order].CustomerId = Customer.CustomerId 
	JOIN OrdersDetail OrderDetail   ON [Order].OrderId = OrderDetail.OrderId 
GROUP BY 
 ProductId
,CustomerName

# Query a view

In [ ]:
SELECT * 
FROM vw_product_quantity_sold_to_customer

# Delete a view
- Has **no** implications on the underlying data, it's just... a saved  `SELECT` statement.

# Alter a view
"But what if I want to change the `SELECT` statement in my view?,,

"Good question, billy!"

In [ ]:
ALTER VIEW vw_product_quantity_sold_to_customer AS
-- 
SELECT 
 ProductId
,CustomerName
,SUM(Quantity) AS 'Total Quanties' -- Change is here! 
FROM Customer 
    JOIN Orders [Order]             ON [Order].CustomerId = Customer.CustomerId 
	JOIN OrdersDetail OrderDetail   ON [Order].OrderId = OrderDetail.OrderId 
GROUP BY 
 ProductId
,CustomerName

# Every complex problem, can be split into multiple smaller problems.

## The monolith
What does the following subquery do?

In [ ]:
-- Insert Subquery here.

# The Purchases

In [ ]:
USE Xtreme;
GO
CREATE VIEW vw_sale AS 
    SELECT 
    FORMAT(OrderDate,'yyyy-MM') AS 'Month'
    ,SUM(OrderAmount)  AS 'Total' 
    FROM Orders 
    GROUP BY FORMAT(OrderDate,'yyyy-MM');

# The Sales

In [ ]:
USE Xtreme;
GO;
CREATE VIEW vw_purchase AS
    SELECT
    FORMAT(OrderDate,'yyyy-MM') AS 'Month' 
    ,SUM(Product.Price * Purchase.UnitsOnOrder) AS 'Total' 
    FROM Purchases Purchase
        JOIN Product on Purchase.ProductId = Product.ProductId
    GROUP BY FORMAT(OrderDate,'yyyy-MM');

# The query

In [ ]:
SELECT 
 ISNULL(sale.Month, purchase.month) AS 'Month'
,ISNULL(sale.Total,0) - ISNULL(purchase.total,0) AS 'MARGIN'
FROM vw_sale sale
    FULL JOIN vw_purchase purchase on sale.Month = Purchase.Month
ORDER BY 1; -- AHA, an ORDER BY clause!


"A View is only used to read data"
- Nope, you can update, delete rows using a view.
- But there are some restriction:
  - Has no distinct or top clause in the select statement
  - Has no statistical functions in the select statement
  - Has no calculated value in the select statement
  - Has no group by in the select statement
  - Does not use a union
  - You can only update one table at once

"Then why would you use it?"
- Security
- Ease of use

# Updatable View

In [37]:
CREATE VIEW vw_updatable AS
SELECT
 ProductId
,ProductName
FROM Product
WHERE ProductName LIKE 'Guardian%'
-- WITH CHECK OPTION

In [38]:
SELECT * FROM vw_updatable

In [42]:
BEGIN TRANSACTION
    INSERT INTO vw_updatable(ProductId, ProductName)
    VALUES(123456789,'Test')
ROLLBACK